The notebook is made to run on kaggle
- Needs glove-840b-300d and infersent 

Adapted From : https://www.kaggle.com/code/jacksoncrow/infersent-demo

## Environment Setup

In [1]:
!pip install --upgrade packaging

     |████████████████████████████████| 40kB 2.2MB/s 
ERROR: allennlp 0.9.0 requires flaky, which is not installed.
ERROR: allennlp 0.9.0 requires responses>=0.7, which is not installed.
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Found existing installation: packaging 19.2
    Uninstalling packaging-19.2:
      Successfully uninstalled packaging-19.2


In [2]:
!pip install datasets --upgrade

     |████████████████████████████████| 368kB 3.4MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 25.6MB 45.3MB/s 
     |████████████████████████████████| 993kB 25.3MB/s 
     |████████████████████████████████| 143kB 40.9MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 276kB 50.3MB/s 
     |████████████████████████████████| 163kB 47.8MB/s 
     |████████████████████████████████| 194kB 43.6MB/s 
     |████████████████████████████████| 163kB 41.1MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=a1970963b0ebff09d0a823717bd58da33e9003684f6167087a1f41a6f0501225
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl
ERROR: allennlp 0.9.0 requires flaky, which is not installed.
ERROR: hyperopt 0.2.2 has requirement networkx==2.2, but you'll have networkx 2.4 which is in

In [3]:
%matplotlib inline

from random import randint
import numpy as np
import torch
import shutil
import string
import nltk.data
import matplotlib
import pandas as pd
import re

matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

In [4]:
# here we need to restructure working directory, so that script imports working properly
shutil.copytree("/kaggle/input/infersent/", "/kaggle/working/infersent")
! mv /kaggle/working/infersent/* /kaggle/working/

In [5]:
%%time

! mkdir encoder
! curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  28.0M      0  0:00:05  0:00:05 --:--:-- 34.0M
CPU times: user 83.5 ms, sys: 33.9 ms, total: 117 ms
Wall time: 7.12 s


## Load Model

In [6]:
model_version = 1
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
W2V_PATH = '/kaggle/input/glove-840b-300d/glove.840B.300d.txt' # change this to your local
VOCAB_SIZE = 1e5  # Load embeddings of VOCAB_SIZE most frequent words
USE_CUDA = True  # Keep it on CPU if False, otherwise will put it on GPU

In [7]:
from models import InferSent
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [8]:
%%time
model = model.cuda() if USE_CUDA else model

model.set_w2v_path(W2V_PATH)

model.build_vocab_k_words(K=VOCAB_SIZE)

Vocab size : 100000.0
CPU times: user 13.4 s, sys: 1.73 s, total: 15.1 s
Wall time: 26.1 s


## Get Data

In [ ]:
from datasets import load_dataset

dataset = load_dataset("recastai/coyo-75k-augmented-captions")

## Encode Sentences

We need to specify input sentences as a list, where each punctuation sign is padded with a space from both sides

In [10]:
sentences = dataset['train']['short_caption']

In [11]:
### converting to a list of strings
sentences = [sentence[0] for sentence in sentences]
sentences_ = [str(sentence) for sentence in sentences]

In [13]:
#use this to add space in the list as appropriate : here the captions have no punctuations
# def add_spaces_around_punctuation(sentences):
#     def add_spaces(text):
#         pattern = r"(?<!\s)([!\"#$%&'()*+,-./:;<=>?@\[\\\]^_`{|}~])"
#         new_text = re.sub(pattern, r" \1", text)
        
#         pattern = r"([!\"#$%&'()*+,-./:;<=>?@\[\\\]^_`{|}~])(?!\s)"
#         new_text = re.sub(pattern, r"\1 ", new_text)
        
#         return new_text
    
#     return [add_spaces(sentence) for sentence in sentences]

#### Applying the model embedings

In [14]:
embeddings = model.encode(sentences_, bsize=128, tokenize=False, verbose=True)
df = pd.DataFrame(embeddings)
torch.cuda.empty_cache()

Nb words kept : 858700/864110 (99.4%)
Speed : 5545.3 sentences/s (gpu mode, bsize=128)


In [15]:
df['prompt'] = sentences 
df.to_csv("infersent_embeddings.csv")

You can access the output using kaggle api\
kaggle kernels output anantjain1223/infersent-demo -p /path/to/dest